In [1]:
import pandas as pd
import re
from tqdm.autonotebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import copy

/tmp/ipykernel_194135/4032043441.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
excel = pd.ExcelFile("./Рейтинг ESG - Отчеты компаний.xlsx")
sheets = {}
for sheet_name in ["E топики", "G топики", "S топики"]:
    sheets[sheet_name] = pd.read_excel(excel, sheet_name=sheet_name)

In [74]:
words = {}
words["G"] = sheets["G топики"].groupby("УНИВЕРСАЛЬНЫЕ ТОПИКИ")["topic_words"].apply(", ".join)
words["S"] = sheets["S топики"].groupby("УНИВЕРСАЛЬНЫЕ ТОПИКИ")["topic_words"].apply(", ".join)
words["G"]

УНИВЕРСАЛЬНЫЕ ТОПИКИ
Безопасность и охрана труда          безопасность, охрана, пасность, работник, труд...
Безопасность продукта                кредитование, опасность, финансовыи, риска, ри...
Благотворительность                  областнои, волонтерство, фонд, спонсорство, фо...
Инвестиции и капитальные вложения    стратегическии, стратегически, инвестиция, лид...
Налоги                               налогообложение, налог, налоговыи, бюджетныи, ...
Обучение и развитие                  образование, выпускник, образоваться, диплом, ...
Оплата труда                         пенсия, пенсионныи, пенсионер, льготныи, работ...
Отношения  потребителями             маркетинг, маркетинговыи, демографическии, дет...
Отношения с потребителями            потребитель, потребительскии, клиент, клиентск...
Отношения с работниками              компенсация, компенсационныи, уплата, выплата,...
Охрана здоровья                      медицина, поликлиника, диспансеризация, медици...
Профсоюзы и коллективн

In [75]:
words["G"] = words["G"].str.split(", ").apply(set).apply(list)
words["S"] = words["S"].str.split(", ").apply(set).apply(list)

In [76]:
words["S"]

УНИВЕРСАЛЬНЫЕ ТОПИКИ
Антикоррупция                         [нарушать, круг, политик, ликвидность, вправе,...
Дивиденды и акционеры                 [одобрение, атомфлот, зачисление, внешнеторгов...
Инновации                             [изобретатель, лицензия, ресурсныи, идентифика...
Лидерство                             [менеджерскии, лицевои, репо, протокол, репута...
Отчетность и прозрачность             [полнота, экспортер, реа, репутационныи, полит...
Предотвращение коррупции              [менеджерскии, внешнеторговыи, повреждение, ад...
Управление рисками                    [банковскии, надежно, предполагаться, недвижим...
Эффективность и производительность    [производство, производитель, экономическии, п...
Name: topic_words, dtype: object

In [77]:
re_dict = {}
sheets_dict = {}
base_dict = {}

for idx, row in words["S"].items():
    sheets_dict[idx] = {r: 1 for r in row}
    base_dict[idx] = {r: 0 for r in row}
    re_dict[idx] = re.compile("(" + "(?!\w)|".join(row) + "(?!\w))")

In [68]:
df = pd.read_csv("../parsing/paragraphs.csv.zip", usecols=["cleaned_text"])
df = df.dropna()
df.head()

/tmp/ipykernel_3299/177660427.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../parsing/paragraphs.csv.zip", usecols=["cleaned_text"])


,cleaned_text
0,ммк
1,социальный отчёт ммк за год магнитогорский мет...
3,содержание
4,обращение председатель совет директор параметр...
5,ммк


In [78]:
topics = []
for text in tqdm(df["cleaned_text"].tolist()):
    paragraph_dict = copy.deepcopy(base_dict)
    found_words = False
    for key, regex in re_dict.items():
        words = regex.findall(text)
        for word in words:
            paragraph_dict[key][word] = 1
            found_words = True
    res = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        topics.append(res | {"max_score": None, "max_topic": None})
        continue
    for key, vector in sheets_dict.items():
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
    
    topics.append(res | {"max_score": max(res.values()), "max_topic": max(res, key=lambda k: res[k])})

  0%|          | 0/2309485 [00:00<?, ?it/s]

In [79]:
df_topic = pd.DataFrame(topics) 
df_topic["max_topic"].value_counts()

Управление рисками                    319196
Эффективность и производительность    287344
Антикоррупция                         217183
Дивиденды и акционеры                 194232
Лидерство                             155653
Предотвращение коррупции              152598
Инновации                              94243
Отчетность и прозрачность              91518
Name: max_topic, dtype: int64

In [80]:
df_topic.to_csv("./topics.csv.zip", index=False)

In [2]:
df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)
df = df.dropna()
topics_e = pd.read_csv("topicsE.csv.zip")
topics_s = pd.read_csv("topicsS.csv.zip")
topics_g = pd.read_csv("topicsG.csv.zip")

/tmp/ipykernel_194135/2681922716.py:1: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)


In [3]:
test = df.join(topics_e, rsuffix="E").join(topics_s, rsuffix="S").join(topics_g, rsuffix="G")
test.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text,Экология,Климат,Энергия,...,Обучение и развитие,Оплата труда,Отношения потребителями,Отношения с потребителями,Отношения с работниками,Охрана здоровья,Профсоюзы и коллективные договоры,Трудовые отношения,max_scoreG,max_topicG
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0.0,1 ММК,ммк,0.000000,0.000000,0.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN,NaN
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...,0.260378,0.160128,0.0,...,0.0,0.08737,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.087370,Оплата труда
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3.0,СОДЕРЖАНИЕ,содержание,0.130189,0.000000,0.0,...,0.0,0.15133,0.141421,0.145095,0.188982,0.0,0.179605,0.178743,0.200000,Безопасность продукта
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...,0.000000,0.000000,0.0,...,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN,NaN
5,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,5.0,3 ММК,ммк,0.000000,0.000000,0.0,...,0.0,0.00000,0.000000,0.102598,0.000000,0.0,0.000000,0.079936,0.102598,Отношения с потребителями


In [5]:
test.to_csv("paragrapgs_with_topics.csv.zip")